<div >
<img src = "figs/ans_banner_1920x200.png" />
</div>

# Caso-taller:  Recomendando Música

El objetivo de este caso-taller es construir un sistema de recomendación de Música utilizando los datos de [Last.fm](https://www.last.fm/) provistos  abiertamente por [grouplens](https://grouplens.org/about/what-is-grouplens/) para: **"avanzar la teoría y la práctica de la computación social mediante la construcción y la comprensión de sistemas *(de recomendación)* utilizados por personas reales".**

Los datos contienen información sobre artistas, usuarios, y las veces que estos escucharon sus canciones. Las bases se encuentran en los `Archivos de Laboratorio` en la carpeta `data`, allí también está disponible un archivo [README](data/readme.txt) que contiene más información sobre las bases.


## Instrucciones generales

1. Para desarrollar el *cuaderno*, primero debe descargarlo.

2. Para responder cada inciso deberá utilizar el espacio debidamente especificado.

3. La actividad será calificada sólo si sube el *cuaderno* de jupyter notebook con extensión `.ipynb` en la actividad designada como "entrega calificada por el personal".

4. El archivo entregado debe poder ser ejecutado localmente por el tutor. Sea cuidadoso con la especificación de la ubicación de los archivos de soporte, guarde la carpeta de datos en el mismo `path` de su cuaderno, por ejemplo: `data`.

## Desarrollo


### 1. Carga de datos 

En la carpeta `data` se encuentran los archivos:

   - `artists.dat`  que contienen el identificador del artista (`id`), nombre (`name`), link a la página del artista en last.fm (`url`), y link a la imagen del usuario (`pictureURL`), vale aclarar que varios de estos links están rotos. 
   - `user_artists.dat`  que contiene identificador del usuario (`userID`), nombre del artista que escuchó (`artistID`), y las veces que los escuchó (`weight`).

Cargue estos datos en su *cuaderno*:

   1. Para la base de artistas seleccione las columnas de identificador de artista (`id`) y nombre (`name`). Renombre estas columnas para poder hacer la unión con la base `user_artists.dat`.
   2. Para la base de usuarios y artistas, renombre las columnas de forma tal que se mantenga la consistencia para unir con la base anterior, y renombre la columna `weight` a `nro_reproducciones`.
   3. Una estas bases.
   

In [10]:
#! pip install plotly
#! pip install nbformat>=4.2.0
#! pip install ipywidgets

In [35]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objs as go
import plotly.subplots as sp
import pandas as pd
import plotly.express as px

In [37]:
# Utilice este espacio para escribir el código.
artists = pd.read_csv('data/artists.dat', sep='\t', usecols=[0, 1], header=0, names=['artistID', 'artistName'])
user_artists = pd.read_csv('data/user_artists.dat', sep='\t', header=0, names=['userID', 'artistID', 'nro_reproducciones'])

users_artists = pd.merge(artists, user_artists, on='artistID')
users_artists

,artistID,artistName,userID,nro_reproducciones
0,1,MALICE MIZER,34,212
1,1,MALICE MIZER,274,483
2,1,MALICE MIZER,785,76
3,2,Diary of Dreams,135,1021
4,2,Diary of Dreams,257,152
...,...,...,...,...
92829,18741,Diamanda Galás,454,301
92830,18742,Aya RL,454,294
92831,18743,Coptic Rain,454,287
92832,18744,Oz Alchemist,454,286


(Utilice este espacio para describir el procedimiento, análisis y conclusiones).

- Hemos cargado los dos archivos con información de usuarios y de los artistas que estos usuarios han escuchado alguna vez.  De acuerdo con el archivo `readme.txt` que describe la base, encontramos que el separador es tabulador, por tanto, tuvimos en cuenta este separador para cargar correctamente las columnas en un *DataFrame* de *pandas*.

- Posteriormente, realizamos las transformaciones a los nombres de las columnas según lo sugerido en el enunciado para facilitar la unión de los dos dataframes por el identificador del usuario.

- Luego, aplicamos el método *merge* para unir los dos dataframes, obteniendo uno solo de **92.834 filas** y **4 columnas**.

### 2. Análisis preliminar. 

En esta sección exploraremos la base. Para ello responda las siguientes preguntas.

#### 2.1 ¿Cuantos usuarios y artistas hay en la base?


In [38]:
f"{users_artists['userID'].nunique()} 🧑 | {users_artists['artistID'].nunique()} 🎵"

'1892 🧑 | 17632 🎵'

Contamos con 1.892 usuarios y 17.632 artistas


#### 2.2 ¿Cuáles es la distribución de probabilidad del consumo por artista? (haga el calculo sin ponderar y ponderando por el numero de reproducciones) ¿Qué podemos inferir a partir de la comparación de ambas?

Esta parte el interés es poder analizar qué tanto se escucha cada artista.  Consideraremos:

- Un primer escenario (sin ponderar) en el que todos los artistas son igualmente relevantes sin tener en cuenta el número de reproducciones, únicamente cuántos usuarios lo escucharon.
- Un segundo escenario (ponderando) en el que los artistas adquieren cierto peso o relevancia según la cantidad de reproducciones que han tenido entre los usuarios.

In [39]:
# Utilice este espacio para escribir el código.
nbins = users_artists['artistID'].nunique()

# Sin ponderar
# Solo tenemos en cuenta cuántos usuarios han "consumido" cada artista
listened_artists_count = users_artists.groupby('artistID')['userID'].nunique().reset_index().rename(columns={'userID': 'veces_escuchado'})

# Para ponderar las reproducciones por artista creamos un dataframe temporal donde tengamos la cantidad de usuarios que lo han escuchado y la cantidad de reproducciones que esos usuarios han realizado de cada artista.
# De esta manera daremos un peso a los artistas de acuerdo a la cantidad de reproducciones
artists_stats = users_artists.groupby('artistID').agg({'userID': 'nunique', 'nro_reproducciones': 'sum'}).reset_index()
artists_stats = artists_stats.rename(columns={'userID': 'num_usuarios', 'nro_reproducciones': 'total_reproducciones'})
artists_stats['reproducciones_ponderadas'] = artists_stats['num_usuarios'] * artists_stats['total_reproducciones']
artists_stats

fig_sin_ponderar = px.histogram(listened_artists_count, x='artistID', y='veces_escuchado', nbins=nbins, title='Distribución sin ponderar del consumo por artista')
fig_ponderada = px.histogram(artists_stats, x='artistID', y='reproducciones_ponderadas', nbins=nbins, title='Distribución ponderada del consumo por artista')

# Para mostrar los dos gráficos juntos...
fig = sp.make_subplots(rows=1, cols=2)
fig.add_trace(fig_sin_ponderar['data'][0], row=1, col=1)
fig.add_trace(fig_ponderada['data'][0], row=1, col=2)

# Personalizar etiquetas de ejes
fig.update_xaxes(title_text='artistID', row=1, col=1)
fig.update_xaxes(title_text='artistID', row=1, col=2)
fig.update_yaxes(title_text='Usuarios que lo han escuchado', row=1, col=1)
fig.update_yaxes(title_text='Ponderado de reproducciones', row=1, col=2)
fig.update_layout(title_text='Distribución del consumo por artista (Sin ponderar reproducciones vs. Reproducciones ponderadas)', showlegend=False)

fig.show()

(Utilice este espacio para describir el procedimiento, análisis y conclusiones).

- Realizamos el cálculo de reproducciones de cada artista sin ponderar, únicamente revisando cuántos usuarios han consumido cada artista.
- Luego obtuvimos el ponderado de reproducciones de cada artista según la cantidad de reproducciones por usuario, de esta manera, logramos dar un peso a cada artista de acuerdo a la cantidad de reproducciones.
- Generámos gráficas en plotly para poder comparar los dos gráficos de una forma más cómoda.

- Qué podemos inferir a partir de la comparación de ambas?
  - No ponderar el consumo de los artistas resulta ser un escenario más equitativo para los artistas pues todos serán considerados con la misma importancia sin ser relevante si han sido escuchado 1 o 1 millón de veces, pero del lado del usuario, esto podría resultar en recomendaciones molestas pues puede sugerir artistas no tan escuchados.
  - Ponderar el consumo de los artistas dando un peso a cada uno según la cantidad de reproducciones, dará un indicio de su popularidad.  Esto permitirá identificar los artistas más influyentes y darles prioridad a esos artistas en una futura recomendación.

#### 2.3 Para el usuario 8 (`userID==8`) ¿cuál es la distribución de reproducción de artistas basado en el número de reproducciones relativas?. Presente sus resultados usando tablas y/o gráficas. ¿Encuentra algún patrón en los artistas que escucha y las veces que reproduce? ¿Podemos decir algo de sus preferencias?


In [43]:
# Utilice este espacio para escribir el código.
user_data = users_artists[users_artists['userID'] == 8]

fig = px.histogram(user_data, x='artistName', y='nro_reproducciones', title=f'Reproducciones por artista para usuario 8')
fig.show()

(Utilice este espacio para describir el procedimiento, análisis y conclusiones).

### 3. Generando Recomendaciones

En esta sección nos interesa generar recomendaciones ***nuevas y relevantes*** para el usuario 8 (`userID==8`). Para ello vamos a generar distintos sistemas de recomendación y comparar las recomendaciones generadas.

#### 3.1. Filtrado colaborativo sencillo: promedios simples.

Usando el promedio simple de reproducciones (sin considerar el número de reproducciones) genere una tabla y/o gráfica con 10 recomendaciones de artistas para este usuario. Explique con cuidado su procedimiento y justifique sus elecciones.

In [ ]:
# Utilice este espacio para escribir el código.

(Utilice este espacio para describir el procedimiento, análisis y conclusiones).

#### 3.2.  Filtrado colaborativo sencillo: promedios ponderados.

Usando el promedio de reproducciones ponderado por `nro_reproducciones` genere una tabla y/o gráfica con 10 recomendaciones de artistas para este usuario. Explique con cuidado su procedimiento y justifique sus elecciones. Compare las recomendaciones con el sistema implementado en el paso anterior.


In [ ]:
# Utilice este espacio para escribir el código.

(Utilice este espacio para describir el procedimiento, análisis y conclusiones).

#### 3.3.  Filtrado colaborativo sencillo: similitud de coseno.

Usando el promedio ponderado de reproducciones genere una tabla y/o gráfica  con 10 recomendaciones de artistas para este usuario. Para generar los pesos utilice la distancia de coseno. Explique con cuidado su procedimiento y justifique sus elecciones. Compare las recomendaciones con el sistema implementado en el paso anterior.

In [ ]:
# Utilice este espacio para escribir el código.

(Utilice este espacio para describir el procedimiento, análisis y conclusiones).

#### 3.4.  Filtrado colaborativo usando SVD


Usando la descomposición en valores singulares (SVD) genere una tabla y/o gráfica  con 10 recomendaciones de artistas para este usuario.  Explique con cuidado su procedimiento y justifique sus elecciones. Compare las recomendaciones con el sistema implementado en los pasos anteriores.


In [ ]:
# Utilice este espacio para escribir el código.

(Utilice este espacio para describir el procedimiento, análisis y conclusiones).

#### 3.5.  Filtrado colaborativo usando Análisis de Canasta de Compra

Usando  el algoritmo `Apriori` genere una tabla y/o gráfica  con 10 recomendaciones de artistas para este usuario.  Explique con cuidado su procedimiento y justifique sus elecciones. Compare las recomendaciones con el sistema implementado en los pasos anteriores. Esto puede tomar mucho tiempo, sea cuidadoso al elegir los hiper-parámetors del modelo, utilice los resultados de las estadísticas descriptivas para elegir sus hier-parámetros, y genere solo reglas con 2 elementos. (Puede también aprovechar los recursos de [Google Colab](https://colab.research.google.com/))


In [ ]:
# Utilice este espacio para escribir el código.

(Utilice este espacio para describir el procedimiento, análisis y conclusiones).

### 4. Recomendaciones generales 

De acuerdo con los resultados encontrados, en su opinión ¿qué procedimiento generó las mejores recomendaciones para este usuario? ¿Cómo implementaría una evaluación objetiva de estas recomendaciones? Justifique su respuesta.

(Utilice este espacio para describir el procedimiento, análisis y conclusiones).